# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

/home/maria/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [262]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2,
                 max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        # Ваш код в 1 строчку
        return 1 - (l_c ** 2 / l_s + r_c ** 2 / r_s).sum(axis=1) / (l_s + r_s)[0]

    def __entropy(self, l_c, l_s, r_c, r_s):
        # Ваш код в 1 строчку
        return -(l_c / l_s*np.log(l_c / l_s) + r_c / r_s*np.log(r_c / r_s))

    def __misclass(self, l_c, l_s, r_c, r_s):
        # Ваш код в 1 строчку
        return 1 - (l_c.max(axis=1) + r_c.max(axis=1)) / (l_s + r_s)[0]

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])  # Ваш код в 1 строчку

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])  # Ваш код в 1 строчку

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)  # Ваш код в 1 строчку

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # подготавливает данные и параметры для создания новых веток
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # выбираются пограничные элементы, принадлежащие разным классам
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # создает массив с данными о кол-ве представителей некоторого класса
        # в непрерывнах подразбиениях общего массива
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
                np.bincount(sorted_y[:cut_size],
                        minlength=class_number)

        # создается (по 2 массива, с начала и с конца),
        # в которых указаны позиции первого элемента из группы,
        # принадлежащей однома классу, и массивы из строк, в ячейках которых указано
        # кол-во представителей разных классов на момент просмотра массива
        # до некоторого элемента
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # ищет лучшее разбиение в соответствии с выбранной мерой неопределенности
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # возвращает разбиение на две ветки
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE *
        # self.NON_LEAF_TYPE *

        # self.tree *
        # self.max_depth *
        # self.sufficient_share *
        # self.min_samples_split *

        # self.get_feature_ids *
        # self.__find_threshold *
        # self.__div_samples *
        # self.__fit_node *

        if (self.max_depth is not None) and (depth > self.max_depth) \
                or y.shape[0] <= self.min_samples_split \
                or ((np.bincount(y).max() / y.shape[0]) >= self.sufficient_share):
            class_number = np.bincount(y)
            self.tree[node_id] = (self.LEAF_TYPE,
                                  class_number.argmax(),
                                  class_number.astype(np.float) / y.size)
            return

        ids = self.get_feature_ids(x.shape[1])

        threshold_array = np.array(list(self.__find_threshold(x[:, i], y) for i in ids))
        best_feature_split = np.argmin(threshold_array[:, 0])
        left_x, right_x, left_y, right_y = self.__div_samples(x,
                                                              y,
                                                              ids[best_feature_split],
                                                              threshold_array[best_feature_split][1])

        if left_y.size == 0 or right_y.size == 0:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.size)
            return

        self.feature_importances_[ids[best_feature_split]] += 1 / (depth*2+1)
        self.tree[node_id] = (self.NON_LEAF_TYPE,
                              best_feature_split,
                              threshold_array[best_feature_split][1])
        self.__fit_node(left_x, left_y, node_id * 2 + 1, depth + 1)
        self.__fit_node(right_x, right_y, node_id * 2 + 2, depth + 1)

        return

    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1])
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= np.linalg.norm(self.feature_importances_)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [263]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [264]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [204]:
%time clf.fit(X_train, y_train)

CPU times: user 3.83 ms, sys: 289 µs, total: 4.12 ms
Wall time: 2.81 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [205]:
%time my_clf.fit(X_train, y_train)

CPU times: user 22 ms, sys: 129 µs, total: 22.1 ms
Wall time: 21.4 ms


## Проверка качества работы на wine

In [206]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8363636363636364

In [207]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8917748917748917

## Подготовка данных Speed Dating Data 

In [247]:
df = pd.read_csv('Speed Dating Data.csv', 
                 encoding='cp1251')

df = df.iloc[:, :97]
df = df.drop([
             'id', 
             'idg',
             'condtn',
             'round',
             'position',
             'positin1',
             'order',
             'partner',
             'age_o',
             'race_o', 
             'pf_o_att',
             'pf_o_sin', 
             'pf_o_int',
             'pf_o_fun',
             'pf_o_amb',
             'pf_o_sha',
             'dec_o',
             'attr_o',
             'sinc_o',
             'intel_o',
             'fun_o',
             'amb_o',
             'shar_o', 
             'like_o',
             'prob_o',
             'met_o',
             'sports',
             'tvsports',
             'exercise',
             'dining', 
             'museums',
             'art', 
             'hiking',
             'gaming', 
             'clubbing', 
             'reading', 
             'tv',
             'theater', 
             'movies', 
             'concerts', 
             'music',
             'shopping',
             'yoga',
             'career',
             'expnum'], axis=1)

df.drop_duplicates('iid').age.isnull().sum()
df = df.dropna(subset=['age'])

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)


feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
        'amb1_1', 'shar1_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
          'shar1_1']].sum(axis=1)

df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
           'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                     'fun1_1', 'amb1_1', 'shar1_1']].T /
                          df.loc[:, 'temp_totalsum'].T).T * 100

feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
        'amb2_1', 'shar2_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                        'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)

df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
           'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                     'fun2_1', 'amb2_1', 'shar2_1']].T /
                          df.loc[:, 'temp_totalsum'].T).T * 100


for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)

df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
            .drop(['gender'], axis=1).dropna()

df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
            .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1).dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12)


df.head()

,iid,gender,pid,match,int_corr,samerace,age,field_cd,mn_sat,tuition,...,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,temp_totalsum
0,1,0,11.0,0,0.14,0,21.0,1.0,-999.0,-999.0,...,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,100.0
1,1,0,12.0,0,0.54,0,21.0,1.0,-999.0,-999.0,...,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,100.0
2,1,0,13.0,1,0.16,1,21.0,1.0,-999.0,-999.0,...,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,100.0
3,1,0,14.0,1,0.61,0,21.0,1.0,-999.0,-999.0,...,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,100.0
4,1,0,15.0,1,0.21,0,21.0,1.0,-999.0,-999.0,...,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,100.0


## Проверка скорости работы на Speed Dating Data 

In [248]:
time clf.fit(X_train, y_train)

CPU times: user 89.5 ms, sys: 4.08 ms, total: 93.6 ms
Wall time: 97.5 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [249]:
time my_clf.fit(X_train, y_train)

CPU times: user 1.33 s, sys: 2.67 ms, total: 1.33 s
Wall time: 1.35 s


## Проверка качества работы на Speed Dating Data

In [211]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.534970238095238

In [212]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5679579565056181

## Задание 3

In [226]:
features_names = df_pair.columns[1:]
head = 10

features = clf.feature_importances_
index = (-clf.feature_importances_).argsort()[0: head]

for i in index:
    print(features_names[i], "     ", features[i])

race       0.41186030233875415
income       0.3965376988052994
go_out       0.17303810993707702
goal       0.018563888918869325
int_corr       0.0
samerace       0.0
age       0.0
field_cd       0.0
mn_sat       0.0
tuition       0.0


In [227]:
features = my_clf.feature_importances_
index = (-my_clf.feature_importances_).argsort()[0: head]

for i in index:
    print(features_names[i], "     ", features[i])

income       0.9162708326722891
race       0.30542361089076303
age       0.18325416653445784
go_out       0.18325416653445784
int_corr       0.0
samerace       0.0
field_cd       0.0
mn_sat       0.0
tuition       0.0
imprace       0.0


## Задание 4

In [114]:
params = {
            # The number of trees in the forest
            'n_estimators': range(10, 100, 5),

             'criterion': ['gini', 'entropy'],
             'max_depth': range(5,10),
             'min_samples_split': range(2,5),
             'min_samples_leaf': range(2,5)
         }

rf_clf = RandomForestClassifier()

random_search = RandomizedSearchCV(rf_clf, param_distributions=params)
random_search.fit(X_train, y_train)
print(random_search.best_params_)

{'n_estimators': 35, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 9, 'criterion': 'entropy'}
